In [7]:
pip install scikit-learn
!wget -c https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!time bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
!time conda install -q -y -c conda-forge rdkit

%matplotlib inline
import sys
import os
sys.path.append('/usr/local/lib/python3.7/site-packages/')

#https://github.com/mordred-descriptor/mordred/blob/develop/README.rst
#molecular descriptor calculator

pip install 'mordred[full]'

    100% |████████████████████████████████| 133kB 3.8MB/s 
  mordred 1.1.2 does not provide the extra 'full'
    100% |████████████████████████████████| 1.7MB 16.7MB/s 
    100% |████████████████████████████████| 51kB 18.4MB/s 
  Running setup.py bdist_wheel for mordred ... done
  Stored in directory: /root/.cache/pip/wheels/a0/c8/af/8ea12bf80e243f46c1b21d77bd4980a65e79f96e4f9ad039a8
  Running setup.py bdist_wheel for networkx ... done
  Stored in directory: /root/.cache/pip/wheels/68/f8/29/b53346a112a07d30a5a84d53f19aeadaa1a474897c0423af91
Successfully built mordred networkx


In [9]:
pip install scikit-multilearn


    100% |████████████████████████████████| 92kB 3.4MB/s 


In [10]:
import tensorflow as tf 
#import deepchem as dc
from sklearn.decomposition import KernelPCA
from sklearn.preprocessing import StandardScaler
from sklearn.multiclass import OneVsRestClassifier
from sklearn.utils import class_weight
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.utils import class_weight
from sklearn.metrics import roc_curve, auc, roc_auc_score, f1_score
from sklearn.model_selection import train_test_split
from skmultilearn.model_selection import iterative_train_test_split
import pickle
from keras.layers import Dense, Activation, Dropout, BatchNormalization, Input
from keras.models import Sequential, Model
from keras import optimizers, regularizers, initializers
from keras.callbacks import ModelCheckpoint, Callback
from keras import backend as K
from keras.optimizers import Adam
import tensorflow as tf
from xgboost import XGBClassifier
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import pandas as pd 
import matplotlib.pyplot as plt
import rdkit
from rdkit import Chem

Using TensorFlow backend.


##Load  the data 
## convert the data from raw data to numeric 
## clean the data 

In [11]:
tox21_df= pd.read_csv('../content/tox21.csv')
print(tox21_df.shape)
tox21_df.head()

(7831, 14)


,NR-AR,NR-AR-LBD,NR-AhR,NR-Aromatase,NR-ER,NR-ER-LBD,NR-PPAR-gamma,SR-ARE,SR-ATAD5,SR-HSE,SR-MMP,SR-p53,mol_id,smiles
0,0.0,0.0,1.0,NaN,NaN,0.0,0.0,1.0,0.0,0.0,0.0,0.0,TOX3021,CCOc1ccc2nc(S(N)(=O)=O)sc2c1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,TOX3020,CCN1C(=O)NC(c2ccccc2)C1=O
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,TOX3024,CC[C@]1(O)CC[C@H]2[C@@H]3CCC4=CCCC[C@@H]4[C@H]...
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,TOX3027,CCCN(CC)C(CC)C(=O)Nc1c(C)cccc1C
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,TOX20800,CC(O)(P(=O)(O)O)P(=O)(O)O


In [0]:

# Get indices of NaN
inds = pd.isnull(tox21_df).any(1).nonzero()[0]
#print (inds)

In [13]:
tox21_df.drop(['mol_id','smiles'], axis=1, inplace=True)
print(tox21_df.shape)
tox21_df.head()

(7831, 12)


,NR-AR,NR-AR-LBD,NR-AhR,NR-Aromatase,NR-ER,NR-ER-LBD,NR-PPAR-gamma,SR-ARE,SR-ATAD5,SR-HSE,SR-MMP,SR-p53
0,0.0,0.0,1.0,NaN,NaN,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [45]:
#tox21_df.dropna(inplace=True)
print(tox21_df.shape)
tox21_df=tox21_df.fillna(0)


(7831, 12)


# Installing rdkit package 

[rdkit](https://www.rdkit.org/docs/Overview.html) is an open source toolkit for cheminformatics 


In [47]:
tox21_df.head()

,NR-AR,NR-AR-LBD,NR-AhR,NR-Aromatase,NR-ER,NR-ER-LBD,NR-PPAR-gamma,SR-ARE,SR-ATAD5,SR-HSE,SR-MMP,SR-p53
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
m = Chem.MolFromSmiles('Cc1ccccc1')

In [16]:
print(m)

In [17]:
mol1=Chem.MolFromSmiles('CO(C)C')
print(mol1)

None


In [0]:
tox21_descriptors_df= pd.read_csv('../content/tox21_descriptors_df.csv',low_memory=False)

# New Section

In [20]:
tox21_descriptors_df.shape

(7831, 1825)

In [26]:
print(tox21_descriptors_df.columns)

Index(['ABC', 'ABCGG', 'nAcid', 'nBase', 'SpAbs_A', 'SpMax_A', 'SpDiam_A',
       'SpAD_A', 'SpMAD_A', 'LogEE_A',
       ...
       'SRW10', 'TSRW10', 'MW', 'AMW', 'WPath', 'WPol', 'Zagreb1', 'Zagreb2',
       'mZagreb1', 'mZagreb2'],
      dtype='object', length=1825)


In [29]:
tox21_descriptors_df.head()

,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,12.395521,10.847181,0,0,19.46281318600621,2.447713486071253,4.716371944676344,19.46281318600621,1.216425824125388,3.704693929148567,...,9.698614,62.626045,258.013284,9.923588,442,22,84.0,97.0,6.256944444444445,3.444444
1,11.370728,10.500427,0,0,19.289890469481737,2.4352613894467936,4.703787940034786,19.289890469481733,1.2859926979654488,3.636116437488342,...,9.553433,61.306823,204.089878,7.558884,353,22,76.0,90.0,5.305555555555555,3.444444
2,16.970296,13.706131,0,0,27.990428923985398,2.620855425231915,5.159573376786184,27.990428923985398,1.3328775678088285,4.026392497471618,...,10.503120,70.933915,288.245316,5.438591,812,44,124.0,158.0,6.430555555555555,4.465278
3,14.308545,13.613603,0,1,24.38888550102035,2.349426052875372,4.698852105750742,24.38888550102035,1.2194442750510175,3.8703926266542332,...,9.653808,52.828148,276.220164,5.754587,830,30,92.0,105.0,8.666666666666666,4.861111
4,8.152948,8.723360,4,0,10.363081100704115,2.4494897427831788,4.8989794855663575,10.363081100704115,0.9420982818821922,3.2803128777834543,...,9.682903,42.374286,205.974526,10.840765,136,18,56.0,64.0,8.1875,2.125000


In [0]:
def coerce_to_numeric(df, column_list):
    df[column_list] = df[column_list].apply(pd.to_numeric, errors='coerce')

In [31]:
coerce_to_numeric(tox21_descriptors_df, tox21_descriptors_df.columns)
tox21_descriptors_df.head()

,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,12.395521,10.847181,0,0,19.462813,2.447713,4.716372,19.462813,1.216426,3.704694,...,9.698614,62.626045,258.013284,9.923588,442,22,84.0,97.0,6.256944,3.444444
1,11.370728,10.500427,0,0,19.289890,2.435261,4.703788,19.289890,1.285993,3.636116,...,9.553433,61.306823,204.089878,7.558884,353,22,76.0,90.0,5.305556,3.444444
2,16.970296,13.706131,0,0,27.990429,2.620855,5.159573,27.990429,1.332878,4.026392,...,10.503120,70.933915,288.245316,5.438591,812,44,124.0,158.0,6.430556,4.465278
3,14.308545,13.613603,0,1,24.388886,2.349426,4.698852,24.388886,1.219444,3.870393,...,9.653808,52.828148,276.220164,5.754587,830,30,92.0,105.0,8.666667,4.861111
4,8.152948,8.723360,4,0,10.363081,2.449490,4.898979,10.363081,0.942098,3.280313,...,9.682903,42.374286,205.974526,10.840765,136,18,56.0,64.0,8.187500,2.125000


In [0]:
tox21_descriptors_df = tox21_descriptors_df.fillna(0)

In [34]:
#inds = pd.isnull(tox21_descriptors_df).any(1).nonzero()[0]
#print (inds)

[]


In [61]:
import numpy as np
tox21_descriptors_df.values


array([[ 12.395521 ,  10.84718  ,   0.       , ...,  97.       ,
          6.2569447,   3.4444444],
       [ 11.370728 ,  10.500426 ,   0.       , ...,  90.       ,
          5.3055553,   3.4444444],
       [ 16.970295 ,  13.706131 ,   0.       , ..., 158.       ,
          6.4305553,   4.4652777],
       ...,
       [ 17.179861 ,  13.8637705,   0.       , ..., 160.       ,
          7.0416665,   4.3055553],
       [ 16.356174 ,  12.977056 ,   0.       , ..., 148.       ,
          6.0902777,   4.1527777],
       [ 20.223017 ,  15.039709 ,   0.       , ..., 179.       ,
          6.361111 ,   5.4444447]], dtype=float32)

In [63]:
tox21_descriptors_df.values.dtype

dtype('O')

In [0]:
tox21_d=np.array(tox21_df.values, dtype=np.float32)
tox21_descriptors_d=np.array(tox21_descriptors_df.values, dtype=np.float32)

In [66]:
tox21_d.dtype
tox21_descriptors_d.dtype

dtype('float32')

In [0]:
x_train, y_train,x_test,y_test=iterative_train_test_split(tox21_df,tox21_descriptors_d,0.2)

In [1]:
x_valid, y_valid=iterative_train_test_splits(x_train, y_train,0.2)

NameError: ignored